In [9]:
import pandas as pd
import tweepy
import requests
from access_token import tokens

In [10]:
client = tweepy.Client( bearer_token=tokens['bearer_token'], 
                        consumer_key=tokens['api_token'], 
                        consumer_secret=tokens['api_token_secret'], 
                        access_token=tokens['access_token'], 
                        access_token_secret=tokens['access_token_secret'], 
                        return_type = requests.Response,
                        wait_on_rate_limit=True)

In [22]:
start_time = "2022-03-15T00:00:00Z"
end_time = "2022-03-15T00:00:00Z"
query = "#trump"
max_num_tweets = 100

# tweets = client.search_all_tweets(query=query, start_time=start_time, end_time=end_time, max_results=max_num_tweets, tweet_fields=['context_annotations', 'created_at'])
# tweets = client.search_recent_tweets(query=query, start_time=start_time, max_results=max_num_tweets)
tweets = client.search_recent_tweets(query=query, max_results=max_num_tweets)

tweets_dict = tweets.json() 
tweets_data = tweets_dict['data'] 
df = pd.json_normalize(tweets_data) 

In [23]:
df

,id,text
0,1504573641379487745,"RT @EelcoHoecke: #eenvandaag: ""#trump die vond..."
1,1504573568654344192,Via @RawStory: Madison Cawthorn's rant calling...
2,1504573561901445122,RT @thejoshuablog: Via @crooksandliars: Bannon...
3,1504573561847091215,Funny--DOJ can look into #HunterBiden but not ...
4,1504573501482672137,Random Trump https://t.co/2JoViG0g4V - #random...
...,...,...
95,1504570070613254156,RT @thejoshuablog: Via @NewsCorpse: TRUMP: Me ...
96,1504570069682204680,RT @morenany_1: Ya lo había mencionado #Trump ...
97,1504570056398749696,RT @Repelsteeltje21: #Draad🧵 over de VS-media ...
98,1504570050946154499,RT @cyril_ip: New research published in an Oxf...


In [24]:
df.to_csv("test.csv", quotechar='"', encoding='utf8', index = False, header=True)